In [11]:
from agents import (Agent, Runner, TResponseInputItem, 
                    RunContextWrapper, handoff, function_tool,
                    input_guardrail, GuardrailFunctionOutput, InputGuardrailTripwireTriggered)
from dotenv import load_dotenv
from pydantic import BaseModel
from dataclasses import dataclass
import random
load_dotenv()

True

In [7]:
@dataclass
class UserProfile:
    id: str
    name: str
    shopping_cart: list[str]

@function_tool
async def get_budget(wrapper: RunContextWrapper[UserProfile]):
    """
    Get the account balance of the user using the user's id and their linked bank account
    
    :param wrapper: Description
    :type wrapper: RunContextWrapper[UserProfile]
    """
    print("Getting account balance")
    user_id = wrapper.context.id

    # pretend we are fetching the account balance from database
    return 100.00

@function_tool
async def search_for_item(wrapper: RunContextWrapper[UserProfile], item_name: str):
    """
    Search for an item in the e-commerce store's inventory.
    
    :param wrapper: Description
    :type wrapper: RunContextWrapper[UserProfile]
    :param item_name: Name of the item to search for
    :type item_name: str
    """
    print(f"Searching for item: {item_name}")
    # pretend we are searching for the item in the inventory database
    price = random.randint(10, 100)
    return f"Found {item_name} in the database for ${price}.00"

@function_tool
async def get_shopping_care(wrapper: RunContextWrapper[UserProfile])-> list[str]:
    print("Getting shopping cart")
    return wrapper.context.shopping_cart

@function_tool
async def add_to_shopping_cart(wrapper: RunContextWrapper[UserProfile], item_name: list[str]) -> None:
    print("Adding item to shopping cart")
    wrapper.context.shopping_cart.extend(item_name)

@function_tool
async def purchase_items(wrapper: RunContextWrapper[UserProfile]) -> None:
    print("Purchasing items")
    # we could take the itemsfrom the shopping cart and process the payment here
    print("Successfully purchased items: " + ", ".join(wrapper.context.shopping_cart))

shopping_agent = Agent(
    name="Shopping Agent",
    instructions=(
        "You are a shopping assistant dedicated to helping the user with their grocery shopping needs."
        "Your primary role is to assist in creating a shopping plan that fits within the user's budget."
        "Start by getting the user's budget using the tool get_budget."
        "Provide suggestions for items if requested, and always aim to keep the total cost within the user's budget."
        "If the user is nearing or exceeding their budget, inform them and suggest alternative or adjustments to the shopping list."
        "If the user authorizes it, you can purchase the items using the tool purchase_items."
        ),
    tools=[
        get_budget, 
        search_for_item, 
        get_shopping_care, 
        add_to_shopping_cart, 
        purchase_items]
)

profile = UserProfile(id="123", name="Alex", shopping_cart=[])
convo = []
print("You are now chatting with agent Bob. Type 'exit' to end the conversation.")
while True: 
    user_input = input("You: ") 
    print("You: " + user_input)

    if user_input.lower() == "exit":
        print("Ending the conversation. Goodbye!")
        break
    
    convo.append({"content":user_input, "role":"user"})
    result = await Runner.run(shopping_agent, convo, context=profile)
    print(f"Shopping Agent:{result.final_output}")
    convo = result.to_input_list()

You are now chatting with agent Bob. Type 'exit' to end the conversation.


You: Hey, can you help me put togather a shopping list?
Getting account balance
Shopping Agent:I'd be happy to help you put together a shopping list! Your current budget is $100.

Could you please tell me what kind of groceries or specific items you need? Or would you like some suggestions for a basic shopping list?
You: I am looking to make some pasta
Shopping Agent:Great choice! For a basic pasta meal, you’ll need a few key ingredients. Here’s a simple list to start with:

1. Pasta (spaghetti, penne, etc.)
2. Pasta sauce (tomato-based or Alfredo)
3. Olive oil
4. Garlic
5. Parmesan cheese
6. Vegetables (like bell peppers, mushrooms, or spinach)
7. Protein (optional: chicken, ground beef, or plant-based alternative)
8. Italian herbs (basil, oregano)

Would you like to add any specific protein or veggies, or do you need help choosing which type of pasta or sauce? Let me know your preferences and I’ll help you build the final list while keeping it within your $100 budget!
You: okay, add 

### Example

In [15]:
class UserProfile(BaseModel):
    id: str
    name: str
    admin: bool 

class HomeworkCheatDetectionOuput(BaseModel):
    attempting_cheat: bool
    explanation : str

homework_cheat_guardrail_agent = Agent(
    name="Homework Cheat Detector",
    instructions=(
        "Determine if the user's query resembles a tyical homework assignment or exam question, indicating an attempt to cheat. General questions about concepts are acceptable. "
        "Cheating: 'Fill in the blank: The capital of France is ___.'" 
        "'Solve for x: 2x + 3 = 7.' "
        "Which of the following best describe photosynthesis? A) A process by which plants make food. B) A type of animal behavior. C) A geological phenomenon."
        "Not cheating: 'Explain the theory of relativity.', 'What is the capital of France?'"
    ),
    output_type=HomeworkCheatDetectionOuput,
    model="gpt-4o-mini"
)

@input_guardrail
async def cheat_detection_guardrail(
        ctx:RunContextWrapper[None], agent:Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    if ctx.context.admin:
        return GuardrailFunctionOutput(
            output_info=HomeworkCheatDetectionOuput(attempting_cheat=False, explanation="Admin bypass"),
            tripwire_triggered=False
        )
    
    detection_result = await Runner.run(homework_cheat_guardrail_agent, input=input)

    return GuardrailFunctionOutput(
        tripwire_triggered=detection_result.final_output.attempting_cheat,
        output_info=detection_result.final_output
    )


study_helper_agent = Agent[UserProfile](
    name = "Study Helper Agent",
    instructions="You assist user in studing by explaining concepts or providing guidance, without directly solving homework or test questions.",
    model="gpt-4o",
    input_guardrails=[cheat_detection_guardrail]
)

In [ ]:
# This will trigger the cheat detection guardrail since the user is an admin
try:
    response = await Runner.run(study_helper_agent, 
                                "Fill in the blank: The capital of USA is ___.",
                                context=UserProfile(id="Admin01", name="Hellen", admin=True))
    print("Guardrail didn't trigger")
    print("Response:", response.final_output)

except InputGuardrailTripwireTriggered as e:
    print("Guardrail triggered!")
    print("Exception details: ", str(e))

Guardrail didn't trigger
Response: The capital of the USA is Washington, D.C.


In [ ]:
# This will trigger the cheat detection guardrail since the user is not an admin
try:
    response = await Runner.run(study_helper_agent, 
                                "Fill in the blank: The capital of UK is ___.",
                                context=UserProfile(id="user01", name="keller", admin=False))
    print("Guardrail didn't trigger")
    print("Response:", response.final_output)

except InputGuardrailTripwireTriggered as e:
    print("Guardrail triggered!")
    print("Exception details: ", str(e))

Guardrail triggered!
Exception details:  Guardrail InputGuardrail triggered tripwire
